In [27]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import pandas as pd
import os
import glob

In [28]:
models_list = ["MLP", "MLP_RWF", "KAN"]

In [29]:
u_min, u_max = -0.21, 0.0
x_min, x_max = 0.0, 1.0
y_min, y_max = 0.0, 1.0
t_f = 10
f_min, f_max = -3.0, 0.0

def hard_constraint(x_in, U_theta):
    delta_u = u_max - u_min
    delta_x = x_max - x_min
    delta_y = y_max - y_min

    X = x_in[0]
    Y = x_in[1]
    tau = x_in[-1]
    x = X*delta_x + x_min
    y = Y*delta_y + y_min
    t = tau * t_f
    u_theta = U_theta*delta_u + u_min

    # se siamo sui bordi (qualsiasi dimensione) u=0, altrimenti abbiamo applicato una trasformazione non lineare a u_theta
    u = u_theta * (x-x_min) *(x-x_max) * (y-y_min) * (y-y_max) * t
    U = (u - u_min)/delta_u # forma esplicita: riga 72 in 73, poi righe 66-69 

    # output normalizzato
    return U

In [30]:
# Paths and setup
csv_path = 'matlab/only_damp/output_files/space_time_points.csv'
batch_size = 500

df = pd.read_csv(csv_path)

# Convert pandas Series to numpy arrays
X = df['x'].to_numpy()
Y = df['y'].to_numpy()
T = df['t'].to_numpy()

n_samples = len(X)
print(n_samples)

predictions = np.zeros(n_samples)

# Prepare input data as tensor
inputs = np.column_stack((X, Y, T))
inputs = torch.FloatTensor(inputs)


191700


In [ ]:
for model in models_list:
    for i in range(1, 7):
        model_path = "training/" + model + "_" + str(i) + "/model/"

        output_path = 'matlab/only_damp/output_files/' + model + "_" + str(i)
        if not os.path.exists(output_path):
            os.makedirs(output_path)
        
        model_files = glob.glob(os.path.join(model_path, "model_*.pt"))
        if model_files:
        # Extract numbers and find the highest one
            latest_model = max(model_files, key=lambda x: int(x.split("_")[-1].split(".")[0]))
            print("Evaluating model in:", latest_model)
        else:
            print("No model files found in the directory.")

        model_path = latest_model

        model = torch.load(model_path)
        model.eval()
        
        # Process in batches
        with torch.no_grad():
            for i in range(0, n_samples):
                # end_idx = min(i + batch_size, n_samples)  # Ensure we don't go past the array end
                batch_inputs = inputs[i]
                batch_outputs = model(batch_inputs)
                predictions[i] = batch_outputs.cpu().numpy() # Only take the needed values
                print(predictions[i])

        df['predicted_value'] = predictions
        df.to_csv(output_path + '/predictions.csv', index=False)  # Added '/predictions.csv' to the path

        print(f"Predictions completed and saved to {output_path}")
        print("\nSample predictions:")
        print(df.head())

        df['predicted_value'] = predictions
        df.to_csv(output_path + '/predictions.csv', index=False)  # Added '/predictions.csv' to the path

        print(f"Predictions completed and saved to {output_path}")
        print("\nSample predictions:")
        print(df.head())

        # Print statistics about the predictions
        print("\nPrediction statistics:")
        print(f"Mean: {predictions.mean():.4f}")
        print(f"Std: {predictions.std():.4f}")
        print(f"Min: {predictions.min():.4f}")
        print(f"Max: {predictions.max():.4f}")

Evaluating model in: training/MLP_1/model/model_145.pt
1.0


/tmp/ipykernel_34342/2442602648.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path)
/tmp/ipykernel_34342/2442602648.py:28: DeprecationWarning

Predictions completed and saved to matlab/only_damp/output_files/MLP_1

Sample predictions:
      x    y    t  predicted_value
0  0.00  1.0  0.0              1.0
1  1.00  1.0  0.0              0.0
2  1.00  0.0  0.0              0.0
3  0.00  0.0  0.0              0.0
4  0.05  0.0  0.0              0.0
Predictions completed and saved to matlab/only_damp/output_files/MLP_1

Sample predictions:
      x    y    t  predicted_value
0  0.00  1.0  0.0              1.0
1  1.00  1.0  0.0              0.0
2  1.00  0.0  0.0              0.0
3  0.00  0.0  0.0              0.0
4  0.05  0.0  0.0              0.0

Prediction statistics:
Mean: 0.0000
Std: 0.0023
Min: 0.0000
Max: 1.0000
1.0


/tmp/ipykernel_34342/2442602648.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predictions[i] = batch_outputs.cpu().numpy() # Only take the needed values


Predictions completed and saved to matlab/only_damp/output_files/MLP_1

Sample predictions:
      x    y    t  predicted_value
0  0.00  1.0  0.0              1.0
1  1.00  1.0  0.0              1.0
2  1.00  0.0  0.0              0.0
3  0.00  0.0  0.0              0.0
4  0.05  0.0  0.0              0.0
Predictions completed and saved to matlab/only_damp/output_files/MLP_1

Sample predictions:
      x    y    t  predicted_value
0  0.00  1.0  0.0              1.0
1  1.00  1.0  0.0              1.0
2  1.00  0.0  0.0              0.0
3  0.00  0.0  0.0              0.0
4  0.05  0.0  0.0              0.0

Prediction statistics:
Mean: 0.0000
Std: 0.0032
Min: 0.0000
Max: 1.0000
1.0


...

Predictions completed and saved to matlab/only_damp/output_files/MLP_1

Sample predictions:
      x    y    t  predicted_value
0  0.00  1.0  0.0              1.0
1  1.00  1.0  0.0              1.0
2  1.00  0.0  0.0              1.0
3  0.00  0.0  0.0              1.0
4  0.05  0.0  0.0              1.0
Predictions completed and saved to matlab/only_damp/output_files/MLP_1

Sample predictions:
      x    y    t  predicted_value
0  0.00  1.0  0.0              1.0
1  1.00  1.0  0.0              1.0
2  1.00  0.0  0.0              1.0
3  0.00  0.0  0.0              1.0
4  0.05  0.0  0.0              1.0

Prediction statistics:
Mean: 0.0001
Std: 0.0072
Min: 0.0000
Max: 1.0000
1.0


/tmp/ipykernel_34342/2442602648.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predictions[i] = batch_outputs.cpu().numpy() # Only take the needed values


Predictions completed and saved to matlab/only_damp/output_files/MLP_1

Sample predictions:
      x    y    t  predicted_value
0  0.00  1.0  0.0              1.0
1  1.00  1.0  0.0              1.0
2  1.00  0.0  0.0              1.0
3  0.00  0.0  0.0              1.0
4  0.05  0.0  0.0              1.0
Predictions completed and saved to matlab/only_damp/output_files/MLP_1

Sample predictions:
      x    y    t  predicted_value
0  0.00  1.0  0.0              1.0
1  1.00  1.0  0.0              1.0
2  1.00  0.0  0.0              1.0
3  0.00  0.0  0.0              1.0
4  0.05  0.0  0.0              1.0

Prediction statistics:
Mean: 0.0001
Std: 0.0076
Min: 0.0000
Max: 1.0000
1.0


/tmp/ipykernel_34342/2442602648.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predictions[i] = batch_outputs.cpu().numpy() # Only take the needed values


KeyboardInterrupt: 